# 1) Titolo e obiettivi

Lezione 34: Document Intelligence - Estrarre informazioni strutturate da documenti

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, cos'è Document Intelligence | 5 min |
| 2 | Teoria: pipeline di estrazione, approcci | 15 min |
| 3 | Schema mentale: workflow estrazione | 5 min |
| 4 | Demo: regex, normalizzazione, DataFrame output | 30 min |
| 5 | Esercizi guidati | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Definire una **pipeline di estrazione** documenti | Sai identificare i campi da estrarre? |
| 2 | Usare **regex** per campi strutturati | Sai estrarre email, date, importi? |
| 3 | **Normalizzare** i valori estratti | Sai convertire date in ISO, importi in float? |
| 4 | **Combinare** regex e NER | Sai quando usare quale? |
| 5 | **Strutturare output** in DataFrame | Sai organizzare le estrazioni? |

---

## L'idea centrale: cos'è Document Intelligence

```
INPUT (documento):                           OUTPUT (dati strutturati):

┌────────────────────────────────┐           ┌──────────────────────────────┐
│ FATTURA N. 123                 │           │ campo       │ valore         │
│ Data: 15/02/2024               │           ├─────────────┼────────────────┤
│ Cliente: Mario Rossi           │    →      │ num_fattura │ 123            │
│ Email: mario@example.com       │           │ data        │ 2024-02-15     │
│ Importo: EUR 1.250,50          │           │ cliente     │ Mario Rossi    │
│ Scadenza: 30/03/2024           │           │ email       │ mario@...      │
└────────────────────────────────┘           │ importo     │ 1250.50        │
                                             └──────────────────────────────┘
        Testo semi-strutturato          →           Tabella strutturata
```

**Document Intelligence = trasformare testo in dati azionabili.**

---

## Tipi di documenti e approcci

| Tipo documento | Complessità | Approccio consigliato |
|----------------|-------------|----------------------|
| **Fatture** | Medio | Regex + normalizzazione |
| **Email** | Basso | Header parsing + regex |
| **Contratti** | Alto | NER + regex + ML |
| **Form OCR** | Alto | Layout analysis + regex |
| **Log files** | Basso | Regex con pattern fissi |

---

## Pipeline Document Intelligence

```
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│  Documento   │ →  │   Pulizia    │ →  │  Estrazione  │ →  │Normalizzazione│
│  (PDF/text)  │    │  encoding    │    │  regex/NER   │    │  date, float │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
                                                                    │
                                                                    ▼
                                                           ┌──────────────┐
                                                           │  Validazione │
                                                           │  formato     │
                                                           └──────────────┘
                                                                    │
                                                                    ▼
                                                           ┌──────────────┐
                                                           │  DataFrame   │
                                                           │  output      │
                                                           └──────────────┘
```

---

## Pattern regex comuni per documenti

| Campo | Pattern | Esempio match |
|-------|---------|---------------|
| **Email** | `[\w.+-]+@[\w-]+\.[\w.-]+` | mario.rossi@example.com |
| **Data IT** | `\d{1,2}/\d{1,2}/\d{2,4}` | 15/02/2024 |
| **Importo EUR** | `EUR?\s*[\d.,]+` | EUR 1.250,50 |
| **Codice Fiscale** | `[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]` | RSSMRA80A01H501A |
| **Telefono IT** | `(?:\+39\s?)?\d{2,4}[\s.-]?\d{6,7}` | +39 02 1234567 |
| **IBAN** | `IT\d{2}[A-Z]\d{22}` | IT60X0542811101000000123456 |

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| Regex | Lezioni 30, 33 | Sai usare re.findall? |
| NER | Lezione 33 | Sai estrarre entità con spaCy? |
| Pandas | Lezioni 1-3 | Sai creare DataFrame? |

**Cosa useremo:** re module, pandas DataFrame, opzionalmente spaCy NER.

# 2) Teoria concettuale
- Document intelligence combina NER, regex e parsing per estrarre campi chiave.
- Input: PDF/OCR trasformato in testo, email, log; Output: tabelle con entita' o campi (nome, data, importi).
- Sfide: formati variabili, rumorosi; soluzione: pattern progressivi (regex), fallback manuali, validazioni.


## Scelte di estrazione
- Regex per campi strutturati (email, date, importi). Pro: semplice; Contro: fragile su formati nuovi.
- NER per entita' generiche (PER/ORG/LOC). Pro: copertura; Contro: richiede modelli e lingua corretta.
- Post-processing: normalizzare campi (date ISO, importi float) e deduplicare.


# 3) Schema mentale / mappa decisionale
1. Identifica i campi da estrarre (nome, email, importo, data, indirizzo).
2. Scegli pattern regex e/o NER adeguati.
3. Applica estrazione al testo pulito.
4. Normalizza i valori (formati data/importi) e valida (es. pattern email).
5. Struttura l'output in un DataFrame.
6. Spot-check manuali su campioni se non hai label.


# 4) Sezione dimostrativa
- Demo 1: estrazione di email, date e importi da un testo.
- Demo 2: normalizzazione e costruzione di un DataFrame.
- Demo 3: combinazione con (opzionale) NER se disponibile.


In [ ]:
# Demo 1: estrazione con regex da testo misto
import re
import pandas as pd

text = """
Fattura n. 123 del 15/02/2024
Cliente: Mario Rossi
Email: mario.rossi@example.com
Importo: EUR 1.250,50
Pagamento previsto entro 30/03/2024
"""

emails = re.findall(r"[\w\.]+@[\w\.]+", text)
dates = re.findall(r"\d{1,2}/\d{1,2}/\d{4}", text)
amounts = re.findall(r"\d{1,3}(?:\.\d{3})*,\d{2}", text)
print("Email:", emails)
print("Date:", dates)
print("Importi:", amounts)
assert len(emails) > 0 and len(dates) > 0 and len(amounts) > 0


In [ ]:
# Demo 2: normalizzazione e tabella
from datetime import datetime

# Normalizza importi in float con punto decimale
amounts_float = [float(a.replace('.', '').replace(',', '.')) for a in amounts]
# Normalizza date in formato ISO
norm_dates = [datetime.strptime(d, "%d/%m/%Y").date().isoformat() for d in dates]

rows = []
for email in emails:
    rows.append({
        'email': email,
        'importo': amounts_float[0] if amounts_float else None,
        'data_fattura': norm_dates[0] if norm_dates else None,
        'data_pagamento': norm_dates[1] if len(norm_dates) > 1 else None
    })

df = pd.DataFrame(rows)
print(df)
assert not df.empty


In [ ]:
# Demo 3: (opzionale) uso NER se spaCy disponibile
try:
    import spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    ents = [(ent.label_, ent.text) for ent in doc.ents]
    print("Entita' NER:", ents)
except Exception as e:
    print("spaCy non disponibile o modello non installato, salto NER.")


In [ ]:
# ============================================================
# DEMO 4: Key-Value Extraction per Fatture
# ============================================================
# Estraiamo campi specifici dalle fatture

from typing import Dict, List, Optional
from dataclasses import dataclass

@dataclass
class InvoiceData:
    """Struttura dati per fattura estratta."""
    numero: Optional[str] = None
    data: Optional[str] = None
    cliente: Optional[str] = None
    totale: Optional[str] = None
    iva: Optional[str] = None
    confidence: float = 0.0

class InvoiceExtractor:
    """
    Estrattore di dati da testo fattura.
    Usa regex per campi strutturati.
    """
    
    def __init__(self):
        # Pattern per ogni campo
        self.patterns = {
            'numero': [
                r'(?:fattura|invoice|n\.?|numero|#)\s*[:.]?\s*([A-Z0-9\-/]+)',
                r'(?:n\.?|numero)\s*(?:fattura)?\s*[:.]?\s*([A-Z0-9\-/]+)',
            ],
            'data': [
                r'(?:data|date|del)\s*[:.]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})',
                r'(\d{4}-\d{2}-\d{2})',  # ISO format
            ],
            'totale': [
                r'(?:totale|total|amount|importo)\s*[:.]?\s*(€?\s*[\d.,]+(?:\s*(?:€|euro))?)',
                r'(€\s*[\d.,]+)',
            ],
            'cliente': [
                r'(?:cliente|customer|spett\.?le|to)\s*[:.]?\s*([A-Za-zÀ-ÿ\s&\.]+(?:s\.?r\.?l\.?|s\.?p\.?a\.?)?)',
            ],
            'iva': [
                r'(?:iva|vat)\s*[:.]?\s*([\d.,]+%?|inclusa|esclusa)',
            ],
        }
    
    def extract(self, testo: str) -> InvoiceData:
        """Estrae dati strutturati da testo fattura."""
        result = InvoiceData()
        campi_trovati = 0
        campi_totali = len(self.patterns)
        
        for campo, patterns in self.patterns.items():
            for pattern in patterns:
                match = re.search(pattern, testo, re.IGNORECASE)
                if match:
                    valore = match.group(1).strip()
                    setattr(result, campo, valore)
                    campi_trovati += 1
                    break  # Usa primo match
        
        # Confidence basata su quanti campi trovati
        result.confidence = campi_trovati / campi_totali
        
        return result
    
    def validate(self, data: InvoiceData) -> Dict:
        """Valida i dati estratti."""
        issues = []
        
        if not data.numero:
            issues.append("Numero fattura mancante")
        if not data.data:
            issues.append("Data fattura mancante")
        if not data.totale:
            issues.append("Totale mancante")
        if data.confidence < 0.6:
            issues.append(f"Bassa confidence: {data.confidence:.0%}")
        
        return {
            'valid': len(issues) == 0,
            'issues': issues,
            'requires_review': data.confidence < 0.8
        }

# Test
extractor = InvoiceExtractor()

fatture_test = [
    "FATTURA N. 2024/001 Data: 15/01/2024 Cliente: Rossi S.r.l. Totale: €1.250,00 IVA inclusa",
    "Invoice #456 Date: 2024-03-10 To: Verdi & Co. Amount: €750.00 VAT included",
    "Fattura numero F-789 Spett.le Bianchi SpA Importo €3.500",
]

print("="*60)
print("DEMO 4: EXTRACTION DA FATTURE")
print("="*60)

for i, fattura in enumerate(fatture_test, 1):
    print(f"\n📄 Fattura {i}:")
    print(f"   Testo: '{fattura[:60]}...'")
    
    # Estrazione
    data = extractor.extract(fattura)
    
    print(f"\n   Campi estratti:")
    print(f"      Numero:  {data.numero or 'N/A'}")
    print(f"      Data:    {data.data or 'N/A'}")
    print(f"      Cliente: {data.cliente or 'N/A'}")
    print(f"      Totale:  {data.totale or 'N/A'}")
    print(f"      IVA:     {data.iva or 'N/A'}")
    print(f"      Confidence: {data.confidence:.0%}")
    
    # Validazione
    validation = extractor.validate(data)
    status = "✅ VALIDO" if validation['valid'] else "⚠️ PROBLEMI"
    print(f"\n   Validazione: {status}")
    for issue in validation['issues']:
        print(f"      - {issue}")

In [ ]:
# ============================================================
# DEMO 5: Pipeline Integrata Document Intelligence
# ============================================================
# Combiniamo Classification + Extraction in un'unica pipeline

class DocumentIntelligencePipeline:
    """
    Pipeline completa per Document Intelligence:
    1. Classifica il documento
    2. Applica extractor specifico per tipo
    3. Valida e restituisce risultato strutturato
    """
    
    def __init__(self, vectorizer, classifier):
        self.vectorizer = vectorizer
        self.classifier = classifier
        
        # Extractors per tipo
        self.extractors = {
            'fattura': InvoiceExtractor(),
            'contratto': ContractExtractor(),
            'lettera': LetterExtractor(),
            'report': ReportExtractor(),
        }
    
    def process(self, documento: str) -> Dict:
        """
        Processa un documento end-to-end.
        
        Returns:
            {
                'tipo': str,
                'confidence_classificazione': float,
                'dati_estratti': dict,
                'validation': dict
            }
        """
        # Step 1: Classification
        X = self.vectorizer.transform([documento])
        
        # Predizione con probabilità
        tipo_predetto = self.classifier.predict(X)[0]
        probabilita = self.classifier.predict_proba(X)[0]
        confidence_class = max(probabilita)
        
        # Step 2: Extraction
        if tipo_predetto in self.extractors:
            extractor = self.extractors[tipo_predetto]
            dati = extractor.extract(documento)
            validation = extractor.validate(dati)
        else:
            dati = {}
            validation = {'valid': False, 'issues': ['Tipo non supportato']}
        
        # Step 3: Risultato
        return {
            'tipo': tipo_predetto,
            'confidence_classificazione': confidence_class,
            'dati_estratti': dati if hasattr(dati, '__dict__') else vars(dati) if hasattr(dati, '__dict__') else dati,
            'validation': validation,
            'richiede_review': confidence_class < 0.8 or not validation['valid']
        }

# Extractors semplificati per altri tipi
class ContractExtractor:
    def extract(self, testo):
        @dataclass
        class ContractData:
            parti: Optional[str] = None
            durata: Optional[str] = None
            valore: Optional[str] = None
            confidence: float = 0.0
        
        result = ContractData()
        
        # Pattern semplici
        parti = re.search(r'tra\s+(.+?)\s+e\s+(.+?)(?:\s+durata|\s+decorrenza|$)', testo, re.I)
        if parti:
            result.parti = f"{parti.group(1)} / {parti.group(2)}"
        
        durata = re.search(r'(\d+)\s*(?:mesi|anni|months|years)', testo, re.I)
        if durata:
            result.durata = durata.group(0)
        
        valore = re.search(r'€\s*[\d.,]+', testo)
        if valore:
            result.valore = valore.group(0)
        
        result.confidence = sum([1 for x in [result.parti, result.durata, result.valore] if x]) / 3
        return result
    
    def validate(self, data):
        return {'valid': data.confidence > 0.3, 'issues': []}

class LetterExtractor:
    def extract(self, testo):
        @dataclass
        class LetterData:
            destinatario: Optional[str] = None
            tipo_saluto: Optional[str] = None
            confidence: float = 0.5
        
        result = LetterData()
        
        saluto = re.search(r'(gentile|egregio|spett\.?le|dear)\s+([^,]+)', testo, re.I)
        if saluto:
            result.destinatario = saluto.group(2)
            result.tipo_saluto = saluto.group(1)
            result.confidence = 0.8
        
        return result
    
    def validate(self, data):
        return {'valid': True, 'issues': []}

class ReportExtractor:
    def extract(self, testo):
        @dataclass
        class ReportData:
            tipo_report: Optional[str] = None
            metriche: List = None
            confidence: float = 0.5
        
        result = ReportData()
        result.metriche = []
        
        # Tipo report
        tipo = re.search(r'(trimestrale|settimanale|annual|quarterly|weekly|mensile)', testo, re.I)
        if tipo:
            result.tipo_report = tipo.group(1)
        
        # Metriche numeriche
        metriche = re.findall(r'(\w+)\s*[:=]\s*([\d.,]+[%€]?|€[\d.,]+)', testo)
        result.metriche = metriche[:5]  # Max 5
        
        result.confidence = 0.6 if result.tipo_report else 0.4
        return result
    
    def validate(self, data):
        return {'valid': True, 'issues': []}

# Test pipeline
pipeline = DocumentIntelligencePipeline(vectorizer, classifier)

test_docs = [
    "FATTURA N. 2024/999 Data: 01/04/2024 Cliente: Test Srl Totale: €2.000,00",
    "CONTRATTO tra Alfa SpA e Beta Srl durata 24 mesi valore €100.000",
    "Gentile Dott. Rossi, Le scrivo per confermare l'appuntamento. Cordiali saluti",
    "Report Q1 2024 Fatturato: €1.5M Crescita: +20% Margine: 15%",
]

print("="*60)
print("DEMO 5: PIPELINE INTEGRATA")
print("="*60)

for doc in test_docs:
    print(f"\n{'='*60}")
    print(f"📄 Input: '{doc[:50]}...'")
    
    result = pipeline.process(doc)
    
    print(f"\n   📋 RISULTATO:")
    print(f"      Tipo: {result['tipo'].upper()}")
    print(f"      Confidence: {result['confidence_classificazione']:.1%}")
    print(f"      Review richiesta: {'Sì' if result['richiede_review'] else 'No'}")

# 5) Esercizi svolti (passo-passo)
## Esercizio 34.1 - Estrazione ordini
Obiettivo: estrarre ID ordine, data e importo da un blocco di testo strutturato.


In [ ]:
# Soluzione esercizio 34.1
ordine_text = "Ordine ID: ORD-2024-001 Data: 12/03/2024 Totale: EUR 250,00"
order_id = re.findall(r"ORD-\d{4}-\d{3}", ordine_text)
order_date = re.findall(r"\d{2}/\d{2}/\d{4}", ordine_text)
order_amount = re.findall(r"\d+,\d{2}", ordine_text)
print(order_id, order_date, order_amount)
assert order_id


## Esercizio 34.2 - Validazione campi estratti
Obiettivo: creare una funzione che valida email e importi e segnala quelli non conformi.


In [ ]:
# Soluzione esercizio 34.2
email_pattern = re.compile(r"^[\w\.]+@[\w\.]+$")
amount_pattern = re.compile(r"^\d+[\.,]\d{2}$")

def valida_campi(row):
    return email_pattern.match(row['email']) is not None and amount_pattern.match(str(row['importo']).replace('.', ',')) is not None

print(df.apply(valida_campi, axis=1))


## Esercizio 34.3 - Normalizzazione indirizzi
Obiettivo: dato un testo con indirizzi, estrarre via/citta'/CAP con regex semplici.


In [ ]:
# Soluzione esercizio 34.3
addr_text = "Spedire a: Via Roma 10, 20121 Milano (MI)"
via = re.findall(r"Via\s+[A-Za-z]+\s+\d+", addr_text)
cap = re.findall(r"\d{5}", addr_text)
citta = re.findall(r"[A-Z][a-z]+", addr_text)
print(via, cap, citta)


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **Regex per campi strutturati** | Email, date, codici hanno pattern fissi |
| 2 | **NER per entità generiche** | Nomi, aziende, luoghi sono variabili |
| 3 | **Normalizzazione sempre** | Date ISO, importi float per downstream |
| 4 | **Validazione per qualità** | Pattern valido ≠ valore valido |
| 5 | **Output in DataFrame** | Struttura per analisi e export |

---

## Confronto sintetico: quando usare cosa

| Campo da estrarre | Approccio | Pattern/Strumento |
|-------------------|-----------|-------------------|
| Email | Regex | `[\w.+-]+@[\w-]+\.[\w.-]+` |
| Date (formato fisso) | Regex + dateutil | `\d{1,2}/\d{1,2}/\d{2,4}` |
| Importi | Regex + normalizzazione | `EUR?\s*[\d.,]+` |
| Nomi persone | NER (spaCy) | doc.ents con label PER |
| Aziende | NER + Gazetteer | Lista nota + fallback ML |
| Codici (CF, IBAN) | Regex con validazione | Pattern + checksum |

---

## Workflow normalizzazione

```python
# Date: da DD/MM/YYYY a YYYY-MM-DD (ISO)
def normalize_date(date_str):
    from dateutil.parser import parse
    return parse(date_str, dayfirst=True).strftime('%Y-%m-%d')

# Importo: da "1.250,50" a 1250.50 (float)
def normalize_amount(amount_str):
    clean = amount_str.replace('EUR', '').replace('.', '').replace(',', '.').strip()
    return float(clean)

# Email: validazione basica
def is_valid_email(email):
    import re
    return bool(re.match(r'^[\w.+-]+@[\w-]+\.[\w.-]+$', email))
```

---

## Reference card metodi

| Metodo | Input | Output | Note |
|--------|-------|--------|------|
| `re.findall(pattern, text)` | pattern, testo | lista match | Estrazione multipla |
| `re.search(pattern, text)` | pattern, testo | match object | Primo match |
| `dateutil.parser.parse()` | stringa data | datetime | Parsing flessibile |
| `pd.DataFrame(dict)` | dizionario | DataFrame | Struttura output |
| `spacy.nlp(text).ents` | testo | entità | NER |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Regex troppo stretta | Non cattura varianti | Allarga con `?`, `*`, alternation |
| Regex troppo larga | Falsi positivi | Testa su casi limite |
| No normalizzazione | Date/importi incompatibili | Sempre convertire |
| Ignora encoding | Caratteri speciali rotti | Leggi con encoding corretto |
| Output non strutturato | Difficile da usare | Sempre DataFrame |

---

## Template Document Intelligence

```python
import re
import pandas as pd
from dateutil.parser import parse

def extract_document_fields(text):
    """Estrae campi chiave da un documento testuale."""
    
    # Pattern
    email_pattern = r'[\w.+-]+@[\w-]+\.[\w.-]+'
    date_pattern = r'\d{1,2}/\d{1,2}/\d{2,4}'
    amount_pattern = r'EUR?\s*[\d.,]+'
    
    # Estrazione
    emails = re.findall(email_pattern, text)
    dates = re.findall(date_pattern, text)
    amounts = re.findall(amount_pattern, text)
    
    # Normalizzazione
    dates_norm = [parse(d, dayfirst=True).strftime('%Y-%m-%d') for d in dates]
    amounts_norm = [float(a.replace('EUR', '').replace('.', '').replace(',', '.').strip()) 
                    for a in amounts]
    
    return {
        'emails': emails,
        'dates': dates_norm,
        'amounts': amounts_norm
    }

# Uso
result = extract_document_fields(document_text)
df = pd.DataFrame({
    'tipo': ['email'] * len(result['emails']) + ['data'] * len(result['dates']),
    'valore': result['emails'] + result['dates']
})
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 35 | Information Retrieval | Ricerca semantica in corpus |
| 36 | Knowledge Mining | Costruzione knowledge graph |
| 37+ | Deep Learning, Generativi | Estensioni avanzate |

# 7) Checklist di fine lezione
- [ ] Ho definito i campi da estrarre e scelto pattern adeguati.
- [ ] Ho normalizzato date/importi e validato email/ID.
- [ ] Ho strutturato l'output in DataFrame.
- [ ] Ho valutato la necessita' di NER per entita' non strutturate.
- [ ] Ho documentato limitazioni dei pattern usati.

Glossario
- Document intelligence: estrazione di campi chiave da documenti.
- Regex: espressioni regolari per pattern testuali.
- Normalizzazione: conversione a formati standard (date ISO, importi float).
- Validazione: controlli su formati attesi.
- NER: riconoscimento di entita' nominate.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-02-15 | Creazione: estrazione regex base |
| 1.1 | 2024-02-22 | Aggiunta normalizzazione date/importi |
| 2.0 | 2024-02-28 | Integrato DataFrame output |
| 2.1 | 2024-03-05 | Refactor con pattern comuni |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII pipeline estrazione, tipi documenti table, pattern regex comuni (email, CF, IBAN, telefono), 5 take-home messages, workflow normalizzazione funzioni, template completo Document Intelligence, reference card metodi |

---

## Note per lo studente

Document Intelligence è la base di:

| Applicazione | Documenti | Output |
|--------------|-----------|--------|
| Invoice Processing | Fatture | Tabelle contabili |
| Contract Analysis | Contratti | Clausole, date, parti |
| Email Parsing | Email | Campi strutturati |
| Form Processing | Moduli OCR | Database |

**Pipeline standard:**
1. Input (PDF/text) → 2. Pulizia → 3. Estrazione → 4. Normalizzazione → 5. Validazione → 6. Output

**Prossima tappa:** Lesson 35 - Information Retrieval